In [12]:
import time
import pandas as pd
from pathlib import Path

In [2]:
SOURCE_DIR = Path('../files/output/csv_output/run_results/')
TARGET_DIR = Path('../files/output/csv_output/treat_runs')
VAL_CSV = SOURCE_DIR/'all_direct_routes_3_run.csv'

In [3]:
# 1. Reading raw 2_run csv
df_val = pd.read_csv(VAL_CSV)
df_val

,from_id,to_id,transport_id,price_min_EUR,duration_min
0,280,331,2,16,415
1,280,207,2,23,330
2,107,297,1,46,180
3,297,216,2,5,100
4,107,108,3,16,78
...,...,...,...,...,...
138551,13,187,1,290,810
138552,118,171,1,51,875
138553,163,156,1,130,695
138554,118,78,1,156,390


In [69]:
# set up limits
euro_zone_codes = range(100, 371)
price_limit, duration_limit = 5, 60
# set up conditions
in_euro_zone = 'from_id in @euro_zone_codes and to_id in @euro_zone_codes'
price_less_limit = 'price_min_EUR < @price_limit'
duration_more_limit = 'duration_min > @duration_limit'
# applying filter
filter = df_val.query(in_euro_zone + ' and ' + price_less_limit + ' and ' + duration_more_limit)
filter

,from_id,to_id,transport_id,price_min_EUR,duration_min
96,147,307,3,2,179
232,292,215,3,3,109
424,112,114,2,2,120
652,309,207,2,2,450
850,136,348,3,2,67
...,...,...,...,...,...
135066,348,136,8,3,77
136601,135,141,2,1,1366
137224,354,353,8,4,94
138366,370,140,2,3,70


In [65]:
df_val_filtered = df_val.drop(filter.index, axis=0)
df_val_filtered

,from_id,to_id,transport_id,price_min_EUR,duration_min
0,280,331,2,16,415
1,280,207,2,23,330
2,107,297,1,46,180
3,297,216,2,5,100
4,107,108,3,16,78
...,...,...,...,...,...
138551,13,187,1,290,810
138552,118,171,1,51,875
138553,163,156,1,130,695
138554,118,78,1,156,390


In [13]:
df_val_filtered.to_csv(TARGET_DIR/'all_direct_routes_3_run_filt.csv', index=False)

In [73]:
# 3. Sorting in price ascending order
df = df_val_filtered.sort_values(by=['from_id', 'to_id', 'transport_id', 'price_min_EUR'], 
                                 ignore_index=True,
                                 ascending=True)
df

,from_id,to_id,transport_id,price_min_EUR,duration_min
0,8,9,1,311,340
1,8,10,1,183,690
2,8,11,1,229,545
3,8,12,1,199,1435
4,8,13,1,119,150
...,...,...,...,...,...
138251,663,552,1,674,1185
138252,663,553,1,562,1570
138253,663,554,1,548,1660
138254,663,626,1,506,1475


In [15]:
# 4. Removing duplicates by triples 'from_id', 'to_id', 'transport_id'
df.drop_duplicates(['from_id', 'to_id', 'transport_id'], inplace=True, ignore_index=True)
df

,from_id,to_id,transport_id,price_min_EUR,duration_min
0,8,9,1,311,340
1,8,10,1,183,690
2,8,11,1,229,545
3,8,12,1,199,1435
4,8,13,1,119,150
...,...,...,...,...,...
81916,663,550,1,494,1760
81917,663,552,1,416,990
81918,663,553,1,562,1570
81919,663,554,1,548,1660


In [16]:
df.duplicated(['from_id', 'to_id', 'transport_id']).value_counts()

False    81921
dtype: int64

In [17]:
# 5. Create index for res.csv files and Cutting from_id.csv files
TARGET_DIR.mkdir(parents=True, exist_ok=True)

frames = []
for from_id in df['from_id'].unique():
    
    temp_df = df[df['from_id'] == from_id]
    
    temp_df.index = from_id * 10_000 + range(1, temp_df.shape[0] + 1)
    
    temp_df.index.name = 'path_id'
    
    #temp_df.to_csv(f'{TARGET_DIR}/{from_id}.csv')
    
    frames.append(temp_df)
    
res_df = pd.concat(frames)

res_df.to_csv(f'{TARGET_DIR}/treat3run_filt.csv')

In [18]:
res_df

,from_id,to_id,transport_id,price_min_EUR,duration_min
path_id,,,,,
80001,8,9,1,311,340
80002,8,10,1,183,690
80003,8,11,1,229,545
80004,8,12,1,199,1435
80005,8,13,1,119,150
...,...,...,...,...,...
6630281,663,550,1,494,1760
6630282,663,552,1,416,990
6630283,663,553,1,562,1570
